In [86]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchmetrics import Accuracy
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from scipy.linalg import toeplitz
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

## Load Data and Preprocessing

In [34]:
# Load data
df = pd.read_csv("p2_emg.csv")

In [35]:
# Label mapping (excluding 'rest')
mapg = {
    "Thumb Extension":0,"index Extension":1,"Middle Extension":2,"Ring Extension":3,
    "Pinky Extension":4,"Thumbs Up":5,"Right Angle":6,"Peace":7,"OK":8,"Horn":9,"Hang Loose":10,
    "Power Grip":11,"Hand Open":12,"Wrist Extension":13,"Wrist Flexion":14,
    "Ulnar deviation":15,"Radial Deviation":16
}

df = df[df['label'] != 'rest']
df['label'] = df['label'].map(mapg)

# Normalize EMG features
features = [col for col in df.columns if 'emg' in col]
# Min-Max Scaling to [0, 1]
# min_vals = df[features].min()
# max_vals = df[features].max()
# df[features] = (df[features] - min_vals) / (max_vals - min_vals + 1e-8)


# Parameters
window_size = 100  # adjust based on your data
features = [col for col in df.columns if 'emg' in col]

# Create clean, non-overlapping windows with consistent labels
X = []
y = []

for i in range(0, len(df) - window_size + 1, window_size):
    window = df.iloc[i:i+window_size]
    label_set = window['label'].unique()
    if len(label_set) == 1:  # Only keep windows with a single label
        X.append(window[features].values.astype(np.float32))
        y.append(label_set[0])

X = np.array(X)
y = np.array(y)

# Dataset
class EMGDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X)  # shape: (N, window, channels)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].permute(1, 0), self.y[idx]  # shape: (channels, window)

dataset = EMGDataset(X, y)
print(X.shape)

(2917, 100, 8)


## Feature Extraction

In [36]:
class FeatureExtractor:
    def __init__(self, sampling_rate=1000.0):
        self.sampling_rate = sampling_rate

    def extract_features(self, data: np.ndarray) -> np.ndarray:
        num_samples, _, num_channels = data.shape
        all_features = []

        for sample in data:
            sample_features = []
            for ch in range(num_channels):
                v = sample[:, ch]
                time_feats = self._get_time_features(v)
                freq_feats = self._get_freq_features(v)
                sample_features.extend(time_feats + freq_feats)
            all_features.append(sample_features)

        return np.array(all_features)

    # Time Domain Features
    def _get_time_features(self, v: np.ndarray) -> list:
        abs_v = np.abs(v)
        diff_v = np.diff(v)
        abs_diff_v = np.abs(diff_v)

        return [
            np.mean(abs_v),                             # MAV
            np.var(v),                                  # VAR
            np.sqrt(np.mean(v ** 2)),                   # RMS
            np.sum(abs_diff_v),                         # WL
            np.mean(abs_diff_v),                        # DAMV
            np.std(abs_diff_v),                         # DASDV
            len(np.where(np.diff(np.sign(v)))[0]),      # ZC
            self._MYOP(abs_v, threshold_ratio=0.1),     # MYOP
            np.sum(abs_diff_v > 0.1),                   # WAMP
            self._SSC(v, threshold=0.1),                # SSC
            *self._HIST(v),                             # HIST 
            *self._AR(v)                                # AR 
        ]

    def _MYOP(self, abs_v: np.ndarray, threshold_ratio=0.1):
        threshold = threshold_ratio * np.max(abs_v)
        return (np.sum(abs_v > threshold) / len(abs_v)) * 100

    def _SSC(self, v: np.ndarray, threshold=0.1):
        return np.sum(((np.diff(v, prepend=v[0])[1:-1] * np.diff(v)[1:]) >= threshold).astype(float))

    def _HIST(self, v: np.ndarray, bins=10, normalize=True):
        hist, _ = np.histogram(v, bins=bins, density=normalize)
        return hist.tolist()

    def _AR(self, v: np.ndarray):
        r = np.correlate(v, v, mode='full') / len(v)
        r = r[len(v) - 1:]
        try:
            R = toeplitz(r[:4])
            r_vector = r[1:5]
            coeffs = np.linalg.solve(R, r_vector)
        except np.linalg.LinAlgError:
            coeffs = np.zeros(4)
        return coeffs.tolist()

    # Frequency Domain Features
    def _get_freq_features(self, v: np.ndarray) -> list:
        N = len(v)
        freqs = np.fft.fftfreq(N, d=1/self.sampling_rate)[:N // 2]
        fft_vals = np.fft.fft(v)
        fft_vals = np.abs(fft_vals[:N // 2])
        power_spectrum = fft_vals ** 2

        return [
            self._MNF(power_spectrum, freqs),
            self._MDF(power_spectrum, freqs),
            self._PKF(power_spectrum, freqs),
            self._TTP(power_spectrum),
            *self._SM(power_spectrum, freqs),
            *self._FSR(power_spectrum, freqs),
            self._VCF(power_spectrum, freqs)
        ]

    def _MNF(self, power, freqs):
        return np.sum(freqs * power) / np.sum(power)

    def _MDF(self, power, freqs):
        cumulative = np.cumsum(power)
        total = cumulative[-1]
        median_index = np.where(cumulative >= total / 2)[0][0]
        return freqs[median_index]

    def _PKF(self, power, freqs):
        return freqs[np.argmax(power)]

    def _TTP(self, power):
        return np.sum(power)

    def _SM(self, power, freqs):
        mean_freq = np.sum(freqs * power) / np.sum(power)
        fsm = mean_freq
        ssm = np.sum(power * (freqs - mean_freq) ** 2) / np.sum(power)
        tsm = np.sum(power * (freqs - mean_freq) ** 3) / np.sum(power)
        return fsm, ssm, tsm

    def _FSR(self, power, freqs):
        total = np.sum(power)
        in_band = np.sum(power[(freqs >= 20) & (freqs <= 50)])
        out_band = np.sum(power[(freqs < 20) | (freqs > 50)])
        return in_band / total, in_band / (in_band + out_band)

    def _VCF(self, power, freqs):
        cf = np.sum(freqs * power) / np.sum(power)
        return np.sum(power * (freqs - cf) ** 2) / np.sum(power)


In [37]:
extractor = FeatureExtractor()
features = extractor.extract_features(X)

print(features.shape)

(2917, 272)


In [56]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2, random_state=40)


## Linear Discriminant Analysis

In [92]:
# Create and train the LDA model
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [93]:
# Predict
y_pred = lda.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7722602739726028

Classification Report:
               precision    recall  f1-score   support

           0       0.63      0.89      0.74        36
           1       0.90      0.85      0.88        33
           2       0.96      0.83      0.89        29
           3       0.94      0.94      0.94        35
           4       0.78      0.97      0.86        36
           5       0.85      0.74      0.79        39
           6       0.92      0.73      0.81        45
           7       0.69      0.68      0.68        37
           8       0.50      0.83      0.62        23
           9       0.63      0.76      0.69        34
          10       0.74      0.42      0.54        33
          11       0.97      0.72      0.83        43
          12       0.52      0.79      0.63        34
          13       0.80      0.65      0.71        31
          14       0.88      0.78      0.82        27
          15       0.97      0.76      0.85        41
          16       0.82    

## Extra Trees Classifier

In [96]:
# Create and train the ExtraTreesClassifier model
ETC = ExtraTreesClassifier(n_estimators=500, random_state=42)
ETC.fit(X_train, y_train)

ExtraTreesClassifier(n_estimators=500, random_state=42)

In [97]:
# Predict
y_pred = ETC.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8236301369863014

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.94      0.69        36
           1       0.88      0.85      0.86        33
           2       0.96      0.93      0.95        29
           3       0.91      0.91      0.91        35
           4       0.92      0.94      0.93        36
           5       1.00      0.77      0.87        39
           6       1.00      0.78      0.88        45
           7       0.79      0.70      0.74        37
           8       0.63      0.83      0.72        23
           9       0.66      0.74      0.69        34
          10       0.76      0.48      0.59        33
          11       1.00      0.88      0.94        43
          12       0.70      0.94      0.80        34
          13       0.83      0.65      0.73        31
          14       0.92      0.89      0.91        27
          15       0.95      0.88      0.91        41
          16       0.83    

## LGBM Classifier

In [73]:
# Create and train the LGBM Classifier model
LGBM = LGBMClassifier(random_state=42)
LGBM.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 58435
[LightGBM] [Info] Number of data points in the train set: 2333, number of used features: 272
[LightGBM] [Info] Start training from score -2.806150
[LightGBM] [Info] Start training from score -2.827657
[LightGBM] [Info] Start training from score -2.799083
[LightGBM] [Info] Start training from score -2.849635
[LightGBM] [Info] Start training from score -2.842255
[LightGBM] [Info] Start training from score -2.872108
[LightGBM] [Info] Start training from score -2.918628
[LightGBM] [Info] Start training from score -2.864561
[LightGBM] [Info] Start training from score -2.750964
[LightGBM] [Info] Start training from score -2.834929
[LightGBM] [Info] Start training from score -2.820436
[LightGBM] [Info] Start training from score -2.895098
[LightGBM] [Info] Start training from score -2.834929
[LightGB

LGBMClassifier(random_state=42)

In [74]:
# Predict
y_pred = LGBM.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8082191780821918

Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.94      0.82        36
           1       0.87      0.82      0.84        33
           2       0.87      0.93      0.90        29
           3       0.97      0.91      0.94        35
           4       0.83      0.97      0.90        36
           5       0.97      0.82      0.89        39
           6       0.97      0.67      0.79        45
           7       0.76      0.68      0.71        37
           8       0.59      0.83      0.69        23
           9       0.60      0.76      0.68        34
          10       0.74      0.61      0.67        33
          11       0.85      0.81      0.83        43
          12       0.72      0.85      0.78        34
          13       0.84      0.68      0.75        31
          14       0.79      0.81      0.80        27
          15       0.88      0.85      0.86        41
          16       0.85    

## Random Forest Classifier

In [102]:
# Create and train the Random Forest Classifier model
RFC = RandomForestClassifier(n_estimators=100, random_state=42)
RFC.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [103]:
# Predict
y_pred = RFC.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8287671232876712

Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.94      0.72        36
           1       0.93      0.82      0.87        33
           2       0.97      0.97      0.97        29
           3       0.94      0.86      0.90        35
           4       0.88      0.97      0.92        36
           5       0.94      0.82      0.88        39
           6       1.00      0.82      0.90        45
           7       0.80      0.65      0.72        37
           8       0.55      0.91      0.69        23
           9       0.67      0.76      0.71        34
          10       0.74      0.52      0.61        33
          11       0.97      0.79      0.87        43
          12       0.75      0.88      0.81        34
          13       0.85      0.71      0.77        31
          14       0.92      0.89      0.91        27
          15       0.95      0.88      0.91        41
          16       0.90    

## Results

<img src="results.png" alt="drawing" width="700"/>